## <font color=Crimson size=6 face="宋体">决策树（Decision Tree）</font>
[参考https://blog.csdn.net/weixin_43216017/article/details/87474045](https://blog.csdn.net/weixin_43216017/article/details/87474045)

# 常用的决策树算法有：
## CART（1984年）：用<font color=crimson>基尼系数</font>选取特征，能处理<font color=crimson>连续</font>属性和<font color=crimson>缺失</font>的问题，可以做<font color=crimson>分类和回归</font>
## ID3（1986年）：用<font color=crimson>信息增益</font>选取特征，只适用于<font color=crimson>离散</font>属性，只能做<font color=crimson>分类</font>
## C4.5（1993年）：用<font color=crimson>信息增益率</font>选取特征，能处理<font color=crimson>连续</font>属性和<font color=crimson>缺失</font>的问题，只能做<font color=crimson>分类</font>

## 下面以ID3算法为例展示决策树算法过程
### 算法步骤：
### 1、计算总体信息熵
### 2、计算各属性信息增益
### 3、选取最大信息增益的属性作为决策节点划分样本
### 4、返回1，重新计算总体信息熵，直到总体信息熵为0退出循环

In [5]:
import collections
import os

import ipywidgets as widgets
import pandas as pd
from IPython.display import display,Latex

from dataset_producer import Dataset
import trees
import algo_decision_tree

## 导入数据

In [6]:
DATA_DIR = ".\dataset"
all_data_path = []
for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        all_data_path.append(os.path.join(root, file))

new_dataset=Dataset()
def show_data(path):
    global new_dataset
    new_dataset=Dataset(path=path,haveHead=True)
    display(new_dataset.data)
widgets.interact(show_data,path=all_data_path)

interactive(children=(Dropdown(description='path', options=('.\\dataset\\weather.CSV',), value='.\\dataset\\we…

<function __main__.show_data(path)>

## 信息熵计算公式
### $$H(X)=-\sum_{i=1}^{n}p_ilogp_i$$
### 以最后一列为标签计算总信息熵

In [7]:
myDat=new_dataset.data.to_numpy()
d=collections.Counter(new_dataset.data.iloc[:,-1])
print("以\"%s\"为类标签，对其所有可能取值计数，结果为"%new_dataset.head[-1],d)

def latex_total_ent(d)->str:
    """
    输入计算总信息熵的字典，返回latex string
    """
    ans= '$info(D)='
    denominator=sum(d.values())#分母
    #每个label产生一个多项式
    for i in d:
        numerator=d[i]#分子
        term='-\\frac{%d}{%d}log_2(\\frac{%d}{%d})' %(numerator,denominator,numerator,denominator)
        ans+=term
    ans+= '=%f$' % algo_decision_tree.calcShannonEnt(d)
    return ans

Latex(latex_total_ent(d))

以"Play"为类标签，对其所有可能取值计数，结果为 Counter({'no': 12, 'yes': 12})


<IPython.core.display.Latex object>

## 计算每个属性的条件信息熵和信息增益
### 条件信息熵计算公式
### $$H(Y|X)=\sum_{i=1}^{n}p_iH(Y|X=x_i)$$
### 信息增益计算公式
### $$g(D,A)=H(D)-H(D|A)$$

In [8]:
def latex_condition_ent(ent_info, condition_num:int):
    """
    输入信息熵的字典，返回所有latex string，包含条件信息熵和信息增益
    :param ent_info:List[Entropy,Dict[int,Dict[str,Entropy]]]，列表第一个是总信息熵，第二个条件信息熵
    :return:
    """
    d=ent_info[1][condition_num]
    condition=new_dataset.head[condition_num]
    total_ent=ent_info[0].ent_value

    latex_list=[]
    denominator_list=[]
    ent_list=[]
    for condition_value in d:
        ans= '$info(D|%s==%s)='%(condition,condition_value)
        counter=d[condition_value].ent_dict
        denominator=sum(counter.values())#分母
        denominator_list.append(denominator)
        #每个label产生一个多项式
        for i in counter:
            numerator=counter[i]#分子
            term='-\\frac{%d}{%d}log_2(\\frac{%d}{%d})' %(numerator,denominator,numerator,denominator)
            ans+=term
        ans+= '=%f$' % d[condition_value].ent_value
        ent_list.append(d[condition_value].ent_value)
        latex_list.append(ans)

    #计算总体
    ans= '$info(D|%s)='%condition
    terms=[]
    condition_ent=0
    for i in range(len(denominator_list)):
        term='\\frac{%d}{%d}\\times{%f}'%(denominator_list[i],sum(denominator_list),ent_list[i])
        terms.append(term)
        condition_ent+= denominator_list[i] / sum(denominator_list) * ent_list[i]
    ans+='+'.join(terms)
    ans+='=%f$' % condition_ent
    latex_list.append(ans)

    #计算增益
    ans='$Gain(%s)=%f-%f=%f$'%(condition,total_ent,condition_ent,total_ent-condition_ent)
    latex_list.append(ans)
    return latex_list

ent_info,bestFeat = algo_decision_tree.chooseBestFeatureToSplit(myDat)

def show_condition(property:int):
    latex_list=latex_condition_ent(ent_info,property)
    for latex in latex_list:
        display(Latex(latex))
# show_condition(0)

property_string=new_dataset.head[:-1]
op=[(new_dataset.head[i],i) for i in range(new_dataset.dimension-1)]
widget_property=widgets.Dropdown(
    options=op,
    description='选择属性:',
)
widgets.interact(show_condition,property=widget_property)
display(new_dataset.data)

interactive(children=(Dropdown(description='选择属性:', options=(('Outlook', 0), ('Temperature', 1), ('Humidity', …

,Outlook,Temperature,Humidity,Windy,Play
0,overcast,hot,high,not,no
1,overcast,hot,high,very,no
2,overcast,hot,high,medium,no
3,sunny,hot,high,not,yes
4,sunny,hot,high,medium,yes
5,rainy,mild,high,not,no
6,rainy,mild,high,medium,no
7,rainy,hot,normal,not,yes
8,rainy,cool,normal,medium,no
9,rainy,hot,normal,very,no


## 每轮选择信息增益最大的属性作为分类节点，生成决策树

In [9]:
myTree  = algo_decision_tree.createTree(new_dataset.data)
myTree=trees.TreeNode.init_from_dict(myTree)
myTree.show_tree_graphviz()

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

## 每轮迭代决策树的生成过程

In [ ]:
# debug用全局变量
# show_set=[]
# show_ent_info=[]
def createTree(dataSet:pd.DataFrame):
    """
    在原有的trees.createTree进行改动，通过全局变量记录下每次递归决策树的变化
    :param dataSet:
    :return:
    """
    global show_set,show_ent_info
    # 每次执行前展示一下当前数据集
    show_set+=[dataSet]

    numpy_set=dataSet.to_numpy()
    classList = numpy_set[:,-1]
    if len(collections.Counter(classList))==1:
        show_ent_info+=[(collections.Counter(classList),None)]
        return classList[0]#stop splitting when all of the classes are equal，剩余标签是同一类
    # if len(dataSet[0]) == 1: #stop splitting when there are no more features in dataSet，#只剩下一个数据
    #     return majorityCnt(classList)
    ent_info,bestFeat = algo_decision_tree.chooseBestFeatureToSplit(numpy_set)

    # show_info用于展示每轮如何选择最好的特征
    show_ent_info+=[ent_info]

    bestFeatLabel = dataSet.columns[bestFeat]
    myTree = {bestFeatLabel:{}}
    
    #对于最好特征那一列的每个可取value
    for value in set(numpy_set[:,bestFeat]):
        #过滤
        filt_set=numpy_set[numpy_set[:,bestFeat]==value,:]
        #用过滤的np.ndarray重新构建df，特征和原来相同，再把最好特征那一列删去
        new_set=pd.DataFrame(data=filt_set,columns=dataSet.columns)
        new_set=new_set.drop(bestFeatLabel,axis=1)
        myTree[bestFeatLabel][value] = createTree(new_set)
    return myTree

TODO:使用新写的TreeNode改写所有代码，放弃使用字典的形式生成决策树 \
DONE:treePlotter中传入字典画图对传入字典有要求，传入一个节点的字典，画不出一个节点的树 \
DONE：1.写一个tree类而非字典2.考虑使用graphviz/networkx画出决策树，而非matplotlib

In [ ]:
#初始化全局变量用于可视化
show_set=[]
show_ent_info=[]

myTree=createTree(new_dataset.data)
myTree=trees.TreeNode.init_from_dict(myTree)
show_tree=myTree.show_tree_gradual_graphviz()
#out_set用于展示数据集、out_tree展示树，out_text展示公式
out_set,out_tree,out_text=widgets.Output(),widgets.Output(),widgets.Output()

new_op=[(show_set[0].columns[i],i) for i in range(show_set[0].shape[1]-1)]
new_widget_property=widgets.Dropdown(options=new_op,description='选择属性:')
n_widget = widgets.IntSlider(min=0, max=len(show_set)-1, description='迭代轮数',
                             continuous_update=False,value=0)
def update_dropdown(*args):
    n=n_widget.value
    new_op=[(show_set[n].columns[i],i) for i in range(show_set[n].shape[1]-1)]
    new_widget_property.options=new_op
n_widget.observe(update_dropdown,'value')

def show(n:int,condition_num:int):
    if show_ent_info[n][1]!=None:
        total_ent=latex_total_ent(show_ent_info[n][0].ent_dict)
        condition_ent=latex_condition_ent(ent_info=show_ent_info[n],condition_num=condition_num)
    else:
        total_ent=latex_total_ent(show_ent_info[n][0])
        condition_ent=["剩余的类标签都是同一类，总信息熵为0"]
    with out_set:
        out_set.clear_output()
        display(show_set[n])
    with out_text:
        out_text.clear_output()
        display(Latex(total_ent))
        for i in condition_ent:
            display(Latex(i))
        # display(show_ent_info[n])
    with out_tree:
        out_tree.clear_output()
        display(show_tree[n])

a=widgets.interactive(show,n=n_widget,condition_num=new_widget_property)
display(a)
# display(widgets.HBox([out_set,out_text]))
# display(out_tree)
display(widgets.HBox([out_set,out_text]))
display(out_tree)
# display(widgets.HBox([n_widget,new_widget_property]))